# AI-Based Personalized Workout and Diet Planner

This notebook contains the **machine learning development and experimentation**
for the Personalized Fitness Planner project.

The final application is deployed using Flask.


## Problem Statement

Most fitness applications provide generic workout and diet plans that do not
consider individual student needs, cultural food habits, budget constraints,
or equipment availability.

This project aims to build an AI-based system that generates **personalized,
practical, and budget-friendly workout and diet plans** for students.


## Datasets Used

1. Indian Food Nutrition Dataset
   - Calories, protein, carbohydrates, fats
   - Used for diet recommendation

2. Gym Exercise Dataset
   - Exercise name, muscle group, equipment
   - Used for workout recommendation

3. Student Fitness Dataset (synthetic)
   - BMI, calorie needs, time availability
   - Used for clustering students


In [5]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder


## Data Loading and Preprocessing

The datasets are loaded and cleaned to retain only the
features required for machine learning and recommendation logic.


In [7]:
# Load cleaned datasets
food_df = pd.read_csv("clean_food_data.csv")
workout_df = pd.read_csv("clean_workout_data.csv")
student_df = pd.read_csv("student_fitness_data.csv")

# Display sample rows
food_df.head(), workout_df.head(), student_df.head()


(                          dish  calories  protein  carbs  fats
 0         Hot tea (Garam Chai)     16.14     0.39   2.58  0.53
 1               Instant coffee     23.16     0.64   3.65  0.75
 2               Espreso coffee     51.54     1.75   6.62  2.14
 3                     Iced tea     10.34     0.03   2.70  0.01
 4  Raw mango drink (Aam panna)     35.92     0.16   9.05  0.03,
                 exercise       muscle equipment description
 0         Rickshaw Carry     Forearms     Other     Average
 1       Single-Leg Press   Quadriceps   Machine     Average
 2         Landmine twist   Abdominals     Other     Average
 3       Weighted pull-up         Lats     Other     Average
 4  T-Bar Row with Handle  Middle Back     Other     Average,
    age  gender  height_cm  weight_kg    bmi         goal     diet budget  \
 0   23    male        155         71  29.55     fat_loss      veg    low   
 1   21    male        183         80  23.89  muscle_gain      veg    low   
 2   20  female  

## Feature Engineering

BMI and daily calorie requirements are computed as derived features
to represent the fitness profile of each student.


In [8]:
def calculate_bmi(weight, height_cm):
    """
    Calculates Body Mass Index (BMI)
    """
    height_m = height_cm / 100
    return weight / (height_m ** 2)


def calculate_daily_calories(age, weight, height_cm, gender):
    """
    Estimates daily calorie requirement using BMR formula
    """
    if gender == "male":
        return 10 * weight + 6.25 * height_cm - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height_cm - 5 * age - 161


## Machine Learning Approach

Unsupervised learning is used because there is no labeled data
indicating the "perfect" workout or diet.

K-Means clustering groups students with similar fitness profiles
based on BMI, calorie needs, and time availability.


In [9]:
# Encode categorical variables
encoded_df = student_df.copy()

encoder = LabelEncoder()
encoded_df['goal'] = encoder.fit_transform(encoded_df['goal'])

# Select features for clustering
X = encoded_df[['bmi', 'daily_calories', 'time_per_day']]

# Train K-Means model
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X)

# Predict cluster for a sample student
kmeans.labels_[:5]


array([0, 1, 0, 3, 3], dtype=int32)

## Hybrid Recommendation System

The system uses:
- Machine Learning (K-Means) for high-level strategy selection
- Rule-based logic for personalization and constraints

This hybrid approach ensures recommendations are practical and valid.


In [10]:
def get_cluster_strategy(cluster_id):
    strategies = {
        0: "Fat Loss",
        1: "Muscle Gain",
        2: "General Fitness",
        3: "Healthy Weight Gain"
    }
    return strategies[cluster_id]

get_cluster_strategy(0)


'Fat Loss'

## Final Application

The trained logic and recommendation functions from this notebook
are transferred into a Flask application (`model_utils.py` and `app.py`)
to build a complete web-based system.
